In [1]:
!nvidia-smi

Sun Oct 29 09:49:37 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8    12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install -q numpy
!pip install -q matplotlib
!pip install -q pandas
!pip install -q tensorflow
!pip install -q scikit-learn
!pip install -q pandas-datareader
!pip install -q yfinance
!pip install --upgrade mplfinance
!pip install -q pmdarima
!pip install -q transformers
!pip install -q transformers[torch]
!pip install -q numpy
!pip install -q torch
!pip install -q scikit-learn
!pip3 install -q datasets
!pip install -q pmdarima
# Code Source Note: https://github.com/twopirllc/pandas-ta
!pip install -q pandas_ta
!pip install beautifulsoup4
!pip install feedparser

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 72.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 105.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 87.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (set

In [2]:
import os
import sys
from google.colab import drive
drive.mount('/content/drive')

# Set the working directory for the tasks
SKELETON_DIR = '/content/drive/MyDrive/stock-prediction/Final/Predict'
os.chdir(SKELETON_DIR)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
from dataset import create_predict_datasets
from transformers import pipeline
from keras.models import load_model
from models import intialize_bert_model

from datetime import datetime, timedelta
from predict import predictions
from visualize import plot_predictions

In [4]:
# Model path
lstm_path = '/content/drive/MyDrive/stock-prediction/Final/Training/model/best_model.h5'
finbert_path = '/content/drive/MyDrive/stock-prediction/Final/Training/finbert'

In [5]:
# Initialized Model
lstm_model = load_model(lstm_path) # LSTM Model

finbert_model, finbert_tokenizer = intialize_bert_model(path=finbert_path, num_labels=3) # Bert Model
finbert_nlp = pipeline("text-classification", model=finbert_model, tokenizer=finbert_tokenizer)

In [6]:
tick = 'TSLA'
end_date = datetime.now()
start_date = end_date - timedelta(days=730)

'''Here are the default setting, I wouldn't recommend changing anything, except for multisteps predictions if your model has been trained for it'''
# Predictions
predict_df, actual_prices, sentiment, lstm_past_predictions, avg_predictions, lstm_future_predictions, arima_predictions, arima_conf_int = predictions(lstm_model=lstm_model,
                                                                                                                                                        finbert_nlp=finbert_nlp,
                                                                                                                                                        tick=tick,
                                                                                                                                                        start_predict=start_date.strftime('%Y-%m-%d'),
                                                                                                                                                        end_predict=end_date.strftime('%Y-%m-%d'),
                                                                                                                                                        k=5, #Predicting number of days in the future (n_steps in this case)
                                                                                                                                                        step_size=30,
                                                                                                                                                        start_p=0,
                                                                                                                                                        max_p=5,
                                                                                                                                                        start_q=0,
                                                                                                                                                        max_q=5,
                                                                                                                                                        m=7,
                                                                                                                                                        seasonal=True,
                                                                                                                                                        multisteps=True)

Fetching data from 2021-10-29 to 2023-10-29
Loading Existing Data
Raw data shape: (502, 7)
         Date        Open        High         Low       Close   Adj Close  \
0  2021-10-29  360.619995  371.736664  357.736664  371.333344  371.333344   
1  2021-11-01  381.666656  403.250000  372.886658  402.863342  402.863342   
2  2021-11-02  386.453339  402.863342  382.000000  390.666656  390.666656   
3  2021-11-03  392.443329  405.130005  384.206665  404.619995  404.619995   
4  2021-11-04  411.470001  414.496674  405.666656  409.970001  409.970001   

      Volume  
0   89755200  
1  168146100  
2  128213400  
3  103885500  
4   76192200  
Loading Prepared Data
Processed data shape: (502, 14)
         Date        Open        High         Low       Close   Adj Close  \
0  2021-10-29  360.619995  371.736664  357.736664  371.333344  371.333344   
1  2021-11-01  381.666656  403.250000  372.886658  402.863342  402.863342   
2  2021-11-02  386.453339  402.863342  382.000000  390.666656  390.6666

In [10]:
plot_predictions(tick=tick,
                 predict_df=predict_df,
                 lstm_predictions=lstm_future_predictions,
                 arima_predictions=arima_predictions,
                 avg_predictions=avg_predictions,
                 arima_conf_int=arima_conf_int,
                 sentiment=sentiment,
                 n=50)